In [29]:
import numpy as np
import pandas as pd

### TRANSFORMACIONES ARCHIVO STOCK MIGRATORIO NACIONES UNIDAS

### 1. Ingesta de datos

In [30]:
df_stock = pd.read_excel('Naciones_Unidas_undesa_pd_2020_ims_stock_by_sex_destination_and_origin.xlsx',engine='openpyxl',sheet_name='Table 1',skiprows= 10)

Cambio nombre de algunas columnas

In [31]:
df_stock.columns=['Index','Destination','Notes of destination','Location code of destination', 'Type of data of destination', 'Origin','Location code of origin','1990',
                                                              '1995',
                                                              '2000',
                                                              '2005',
                                                              '2010',
                                                              '2015',
                                                              '2020',
                                                              '1990.1',
                                                              '1995.1',
                                                              '2000.1',
                                                              '2005.1',
                                                              '2010.1',
                                                              '2015.1',
                                                              '2020.1',
                                                              '1990.2',
                                                              '1995.2',
                                                              '2000.2',
                                                              '2005.2',
                                                              '2010.2',
                                                              '2015.2',
                                                              '2020.2'] 

Me quedo solo con las columnas que tienen el total de migrantes y descarto las que tienen migranetes hombre y mujers que son las que tenian la fecha.1 y fecha.2<br>
año: Inventario de migrantes internacionales a mitad de año, ambos sexos combinados. <br>
año.1:Inventario de migrantes internacionales a mitad de año, hombres. <br>
año.2:Inventario de migrantes internacionales a mitad de año, mujeres <br>

In [32]:
df_stock=df_stock[['Index', 'Destination', 'Notes of destination',
       'Location code of destination', 'Type of data of destination', 'Origin',
       'Location code of origin', '1990', '1995', '2000', '2005', '2010',
       '2015', '2020']]

Me fijo si hay duplicados

In [33]:
df_stock_duplicated=df_stock.duplicated()
df_stock_duplicated.sum()

0

Analizo los parametros estadisticos para detectar anomalias o outlieras. Se ve todo dentro de rangos normales

In [34]:
df_stock.describe()

,Index,Notes of destination,Location code of destination,Location code of origin,1990,1995,2000,2005,2010,2015,2020
count,37051.000000,3574.000000,37051.000000,37051.000000,3.705100e+04,3.705100e+04,3.705100e+04,3.705100e+04,3.705100e+04,3.705100e+04,3.705100e+04
mean,18526.000000,19.478176,745.930663,761.146231,2.959657e+05,3.094353e+05,3.304824e+05,3.640804e+05,4.230856e+05,4.775787e+05,5.425222e+05
std,10695.846748,11.194135,732.774287,736.658418,2.596883e+06,2.735540e+06,2.975041e+06,3.306144e+06,3.872671e+06,4.360690e+06,4.922727e+06
min,1.000000,1.000000,4.000000,4.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,9263.500000,10.000000,336.000000,356.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,4.000000e+00,5.000000e+00
50%,18526.000000,18.000000,686.000000,702.000000,1.980000e+02,2.480000e+02,2.960000e+02,3.670000e+02,4.590000e+02,5.220000e+02,6.080000e+02
75%,27788.500000,30.000000,916.000000,920.000000,8.234500e+03,9.629000e+03,1.058800e+04,1.250050e+04,1.456500e+04,1.594900e+04,1.851700e+04
max,37051.000000,41.000000,5501.000000,5501.000000,1.529862e+08,1.612900e+08,1.732306e+08,1.914468e+08,2.209832e+08,2.479586e+08,2.805981e+08


Con la función describe verifico los máximos y mínimmos de cada columna. Observo que los registros maximos son todos del mismo orden de magnitud. 

Voy a agregar el codigo de pais que usa el dataset del banco mundial para despues poder unir lo datos

In [35]:
codigos=pd.read_csv('codigo_pais.csv')

# Genero un diccionario con los valores de los codigos que estan en el archivo all para despues hacer el reemplazo
codigos['country-code']=codigos['country-code'].astype(str)
codigos['country-code']=codigos['country-code'].str.strip()

codigos['alpha-3']=codigos['alpha-3'].astype(str)
codigos['alpha-3']=codigos['alpha-3'].str.strip()

diccionario = dict(zip(codigos['country-code'], codigos['alpha-3']))


# Genero columnas nuevas con los codigos ISO03
df_stock['Location code of destination'] = df_stock['Location code of destination'].astype(str)
df_stock['Location code of origin'] = df_stock['Location code of origin'].astype(str)

df_stock['ISO3_dest'] = df_stock['Location code of destination'].map(diccionario)
df_stock['ISO3_origin'] = df_stock['Location code of origin'].map(diccionario)


In [36]:
df_stock['ISO3_origin'].nunique()

234

ISO3_originMe quedo solo con los registros que tengan el codigo pais (porque habia otros destinos y origen que eran grupos de paises, o regiones y esos no los tengo que tener en cuenta en el dataframe)

In [37]:
df_stock = df_stock[df_stock['ISO3_origin'].notnull() & df_stock['ISO3_dest'].notnull()]

### Genero una funcion que cambie la estructura del dataframe poniendo los años en las filas

In [38]:
def cambio_estructura (df):
    df_melted = pd.melt(df, id_vars=['Destination', 'Notes of destination', 'Location code of destination','ISO3_dest','Type of data of destination', 'Origin', 'Location code of origin','ISO3_origin'], 
    value_vars=['1990', '1995', '2000', '2005', '2010', '2015', '2020'], 
    var_name='Year', value_name='Migrant stock')
    final = df_melted
    return final

In [39]:
Stock_migrantes = cambio_estructura(df_stock)

In [40]:
Stock_migrantes 

,Destination,Notes of destination,Location code of destination,ISO3_dest,Type of data of destination,Origin,Location code of origin,ISO3_origin,Year,Migrant stock
0,Burundi,NaN,108,BDI,B R,Belgium,56,BEL,1990,642
1,Burundi,NaN,108,BDI,B R,Democratic Republic of the Congo,180,COD,1990,36654
2,Burundi,NaN,108,BDI,B R,France*,250,FRA,1990,321
3,Burundi,NaN,108,BDI,B R,Kenya,404,KEN,1990,186
4,Burundi,NaN,108,BDI,B R,Rwanda,646,RWA,1990,221943
...,...,...,...,...,...,...,...,...,...,...
83652,Wallis and Futuna Islands*,41.0,876,WLF,B R,France*,250,FRA,2020,889
83653,Wallis and Futuna Islands*,41.0,876,WLF,B R,French Polynesia*,258,PYF,2020,10
83654,Wallis and Futuna Islands*,41.0,876,WLF,B R,Mayotte*,175,MYT,2020,27
83655,Wallis and Futuna Islands*,41.0,876,WLF,B R,New Caledonia*,540,NCL,2020,1021


### Voy a hacer a union con el dataset de indicadores para tener la tabla que uso en powerbi. 


Para generar esta tabla, leo el dataframe que tiene todos los valores de los indicadores del Banco Mundial.Voy
a hacer un merge para agregar al dataset stock_migrantes, columnas de los valores de los indicadores para el pais de origen y columnnas de los indicadores para el pais destino.

In [41]:
Stock_migrantes['Year']=Stock_migrantes['Year'].astype(int)
Stock_migrantes['ISO3_dest']=Stock_migrantes['ISO3_dest'].astype(str)
Stock_migrantes['ISO3_origin']=Stock_migrantes['ISO3_origin'].astype(str)

In [42]:
merged_df_modelo = pd.read_csv('indicadores_modelo.csv')

In [43]:
merged_df_modelo

,Unnamed: 0,Country Code,Year,Ingresos,Electricidad,Contaminación,Crecimiento PBI,Crecimiento PBI per cápita,Desempleo Hombre,Desempleo mujeres,Gasto PBI,Importaciones,Migracion neta,Desempleados con educación avanzada,Tasa de cambio,Pobreza,Mortalidad
0,0,ABW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.594000
1,1,AFE,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.757511e+09,-90849.0,NaN,NaN,NaN,20.918108
2,2,AFG,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.700000e+07,2606.0,NaN,1.719656e+01,NaN,31.921000
3,3,AFW,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.880000e+09,-847.0,NaN,NaN,NaN,26.363067
4,4,AGO,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.280000e+08,-43749.0,NaN,2.866844e-08,NaN,26.659000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16753,16753,XKX,2022,NaN,NaN,NaN,3.511707,4.924753,NaN,NaN,NaN,NaN,NaN,NaN,9.509155e-01,NaN,NaN
16754,16754,YEM,2022,NaN,NaN,NaN,NaN,NaN,NaN,26.299,NaN,5.715000e+09,NaN,NaN,1.110203e+03,NaN,NaN
16755,16755,ZAF,2022,NaN,NaN,NaN,2.042299,1.187663,27.39,31.426,NaN,1.362080e+11,NaN,15.35,1.635585e+01,NaN,NaN
16756,16756,ZMB,2022,NaN,NaN,NaN,4.744942,1.895518,NaN,6.245,NaN,9.047000e+09,NaN,NaN,1.693759e+01,NaN,NaN


In [44]:
merged_df_modelo['Year']=merged_df_modelo['Year'].astype(int)
merged_df_modelo['Country Code']=merged_df_modelo['Country Code'].astype(str)

In [45]:
# Primera unión con las claves (left Country Code rigth ISO3_origin, Year)
merged_df_modelo_stock1 = pd.merge(Stock_migrantes, merged_df_modelo, left_on=['ISO3_origin', 'Year'], right_on=['Country Code', 'Year'], how='left')

# Segunda unión con las claves (Year, left Country Code, rigth ISO3_dest)
final_merged_df = pd.merge(merged_df_modelo_stock1, merged_df_modelo, left_on=['Year', 'ISO3_dest'], right_on=['Year', 'Country Code'], suffixes=('_origin', '_dest'), how='left')


In [46]:
final_modelo_stock=final_merged_df[['Destination',
       'ISO3_dest','Origin', 'ISO3_origin', 'Year', 'Migrant stock',
       'Ingresos_origin', 'Electricidad_origin',
       'Contaminación_origin', 'Crecimiento PBI_origin',
       'Crecimiento PBI per cápita_origin', 'Desempleo Hombre_origin',
       'Desempleo mujeres_origin', 'Gasto PBI_origin', 'Importaciones_origin',
       'Desempleados con educación avanzada_origin',
       'Tasa de cambio_origin', 'Pobreza_origin', 'Mortalidad_origin',
       'Ingresos_dest', 'Electricidad_dest',
       'Contaminación_dest', 'Crecimiento PBI_dest',
       'Crecimiento PBI per cápita_dest', 'Desempleo Hombre_dest',
       'Desempleo mujeres_dest', 'Gasto PBI_dest', 'Importaciones_dest',
       'Desempleados con educación avanzada_dest',
       'Tasa de cambio_dest', 'Pobreza_dest', 'Mortalidad_dest']]

In [47]:
final_modelo_stock

,Destination,ISO3_dest,Origin,ISO3_origin,Year,Migrant stock,Ingresos_origin,Electricidad_origin,Contaminación_origin,Crecimiento PBI_origin,...,Crecimiento PBI_dest,Crecimiento PBI per cápita_dest,Desempleo Hombre_dest,Desempleo mujeres_dest,Gasto PBI_dest,Importaciones_dest,Desempleados con educación avanzada_dest,Tasa de cambio_dest,Pobreza_dest,Mortalidad_dest
0,Burundi,BDI,Belgium,BEL,1990,642,NaN,100.000000,99.869095,3.137402,...,3.499822,1.421896,0.72,NaN,NaN,231000000.0,NaN,171.255417,NaN,19.067
1,Burundi,BDI,Democratic Republic of the Congo,COD,1990,36654,NaN,NaN,100.000000,-6.568311,...,3.499822,1.421896,0.72,NaN,NaN,231000000.0,NaN,171.255417,NaN,19.067
2,Burundi,BDI,France*,FRA,1990,321,NaN,100.000000,99.774451,2.923935,...,3.499822,1.421896,0.72,NaN,NaN,231000000.0,NaN,171.255417,NaN,19.067
3,Burundi,BDI,Kenya,KEN,1990,186,NaN,NaN,100.000000,4.192051,...,3.499822,1.421896,0.72,NaN,NaN,231000000.0,NaN,171.255417,NaN,19.067
4,Burundi,BDI,Rwanda,RWA,1990,221943,NaN,NaN,100.000000,-2.399291,...,3.499822,1.421896,0.72,NaN,NaN,231000000.0,NaN,171.255417,NaN,19.067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83652,Wallis and Futuna Islands*,WLF,France*,FRA,2020,889,57.17,100.000000,NaN,-7.784586,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83653,Wallis and Futuna Islands*,WLF,French Polynesia*,PYF,2020,10,NaN,100.000000,NaN,-6.999991,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83654,Wallis and Futuna Islands*,WLF,Mayotte*,MYT,2020,27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83655,Wallis and Futuna Islands*,WLF,New Caledonia*,NCL,2020,1021,NaN,100.000000,NaN,-2.400000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Tabla visualizacion Jime

In [48]:
final_modelo_stock.to_csv('./origen_destino_indicadores.csv', index=False)

#### Tabla para visualizaciones Silvina

In [49]:
final_modelo_stock_melted = pd.melt(final_modelo_stock, id_vars=['Destination', 'ISO3_dest', 'Origin', 'ISO3_origin', 'Year'], 
value_vars=['Migrant stock', 'Ingresos_origin', 'Electricidad_origin',
       'Contaminación_origin', 'Crecimiento PBI_origin',
       'Crecimiento PBI per cápita_origin', 'Desempleo Hombre_origin',
       'Desempleo mujeres_origin', 'Gasto PBI_origin', 'Importaciones_origin',
       'Desempleados con educación avanzada_origin', 'Tasa de cambio_origin',
       'Pobreza_origin', 'Mortalidad_origin', 'Ingresos_dest',
       'Electricidad_dest', 'Contaminación_dest', 'Crecimiento PBI_dest',
       'Crecimiento PBI per cápita_dest', 'Desempleo Hombre_dest',
       'Desempleo mujeres_dest', 'Gasto PBI_dest', 'Importaciones_dest',
       'Desempleados con educación avanzada_dest', 'Tasa de cambio_dest',
       'Pobreza_dest', 'Mortalidad_dest'],
var_name='Indicador', value_name='Valor')

In [50]:
final_modelo_stock_melted

,Destination,ISO3_dest,Origin,ISO3_origin,Year,Indicador,Valor
0,Burundi,BDI,Belgium,BEL,1990,Migrant stock,642.0
1,Burundi,BDI,Democratic Republic of the Congo,COD,1990,Migrant stock,36654.0
2,Burundi,BDI,France*,FRA,1990,Migrant stock,321.0
3,Burundi,BDI,Kenya,KEN,1990,Migrant stock,186.0
4,Burundi,BDI,Rwanda,RWA,1990,Migrant stock,221943.0
...,...,...,...,...,...,...,...
2258734,Wallis and Futuna Islands*,WLF,France*,FRA,2020,Mortalidad_dest,NaN
2258735,Wallis and Futuna Islands*,WLF,French Polynesia*,PYF,2020,Mortalidad_dest,NaN
2258736,Wallis and Futuna Islands*,WLF,Mayotte*,MYT,2020,Mortalidad_dest,NaN
2258737,Wallis and Futuna Islands*,WLF,New Caledonia*,NCL,2020,Mortalidad_dest,NaN


In [51]:
final_modelo_stock_melted.to_csv('./origen_destino_indicadores_columna.csv',index=False)